## Step 1: Documentation

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM).

**Documentation:**
 - Product Requirements Document (PRD) generated from a high-level idea.
 - Architecture Decision Records (ADR) including auto-generated technical decisions with their justification
 - Architecture Document including auto-generated UML diagrams (e.g., Component or Sequence diagrams).


**Model Selection:**
Our `utils.py` script is configured to work with multiple AI providers. You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in the `RECOMMENDED_MODELS` dictionary in `utils.py`. For example, to use a Hugging Face model, you could change the line to: `client, model_name, api_provider = setup_llm_client(model_name="meta-llama/Llama-3.3-70B-Instruct")`

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client for our chosen LLM.
- `get_completion()`: To send a prompt to the LLM and get a response.
- `save_artifact()`: To save our generated requirements to a file.

In [ ]:
import sys
import os
import json
import tqdm as notebook_tqdm

project_root = 'c:\\Users\\labadmin\\Documents\\AG-AISOFTDEV'

# Add the project's root directory to the Python path to ensure 'utils.py' can be imported.
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Confirm that utils.py exists in the project root
if not os.path.isfile(os.path.join(project_root, "utils.py")):
    print(f"Warning: utils.py not found in project root: {project_root}")
    # Optionally, you can stop execution here if utils.py is required
    # import sys; sys.exit(1)

# Remove 'utils' from sys.modules if already imported from site-packages
if 'utils' in sys.modules:
    del sys.modules['utils']

from utils import setup_llm_client, get_completion, save_artifact, clean_llm_output
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

app_name="Contracting Visualization Software"
problem_statement = f"""
Contracting Visualization Software is an AI-powered RESTful service application designed to help contracting companies.
We need a tool that will do the following:
1. View and explore historical data of contracts awarded by the US government through an insightful and
    powerful dashboard with various visualizations.
2. The application enables users to analyze trends, access contract details.
3. Its primary focus is on data exploration, visualization, and reporting to support
   strategic decision-making based on awarded contract data.
"""

prd_prompt = f"""
You are an expert AI-enabled product manager and enterprise architect.

Your task is to create a comprehensive Product Requirements Document (PRD) based on the problem statement for the following system:

Problem Statement: {problem_statement}
System: "{app_name}"

The PRD that includes:
1. Executive Overview, Vision and Goals
2. In-scope / Out-of-scope
3. Personas
4. Data Types: Contract Name, Company Awarded, Date Awarded, Location (where contract will be performed), Value of Contract, Length of Contract, Product or service information (NAICS and PSC categories) if available.
5. User stories with story points and acceptance criteria: All acceptance criteria must be measurable and testable with HTTP codes, etc.
6. Functional & non-functional requirements
7. Goals and Success metrics
8. Assumptions/constraints
9. Error handling: Requirements include invalid input, server errors, etc.
10. Dependencies
11. Future considerations
12. Milestones and timeline
13. Security and compliance
14. Error handling
15. Glossary of terms
16. Appendices
17. References: sam.gov, GSA.gov, FPDS.gov

Formatting requirements:
- Use markdown formatting with appropriate headings and subheadings.
- Include tables for personas and user stories.
- Use bullet points for lists of requirements, goals, and other items.
- Ensure clarity and conciseness in language.
Make sure to cover all aspects of the system comprehensively.
The PRD should be detailed and structured to guide the development team effectively.
Use Markdown formatting with appropriate headings and subheadings.
"""

print("*" * 80)
print("Generating PRD for", app_name)
prd_from_template_output = get_completion(prd_prompt, client, model_name, api_provider)
prd_from_template_output = clean_llm_output(prd_from_template_output)
print(prd_from_template_output)
if not prd_from_template_output:
    raise ValueError("No PRD output generated")
else:
    save_artifact(prd_from_template_output, "artifacts/convisoft_prd.md",overwrite=True)
    print("PRD generated successfully, saving to artifacts...")

print("*" * 80)

In [ ]:
# Create the Architecture Decision Records (ADR)
from utils import load_artifact, render_plantuml_diagram
# Load the PRD artifact to reference in the ADRs
prd_md_artifact=load_artifact("artifacts/convisoft_prd.md")

app_description=f"""
Contracting Visualization Software is an AI-powered RESTful service application designed to help contracting companies
view and explore historical data of contracts awarded by the US government through an insightful and
powerful dashboard with various visualizations. The application enables users to analyze trends, access contract details.
Its primary focus is on data exploration, visualization, and reporting to support strategic decision-making based on awarded contract data.
"""

adr_prompt = f"""
You are an expert software architect.

Your task is to create a set of comprehensive Architecture Decision Records (ADR) for the following system, strictly aligned to the Product Requirements Document (PRD):

System PRD: {prd_md_artifact}
System: {app_name}
System Description: {app_description}

Requirements:

* Be verbose and clear to guide the development team.
* First, display a markdown table of Personas and User Stories relevant to the system.
 - The Personas table should include: Persona, Role, Goals, Pain Points.
 - The User Stories table should include: Story ID, Persona, User Story, Story Points, Acceptance Criteria. The acceptance criteria must include measurable and testable items with HTTP codes where applicable.
* Then, for each ADR, include:
    - Context: Background, rationale, relevant requirements, constraints, and goals from the PRD for each ADR.
        • At minimum, cover:
            o Framework choice
            o ML model baseline
            o Feature engineering
            o Database
            o Security
            o Observability
            o Model transparency
            o Awards API design
    - Decision: The chosen solution or approach, clearly stated.
    - Status: Current state (e.g., Proposed, Accepted, Deprecated).
    - Consequences: Impact of the decision including effects on other parts of the system.
    - Justification: A detailed explanation of the reasons for choosing this option, alignment with requirements and goals.
    - Alternatives: Other options considered, with pros and cons.
    - Display the ADRs in numbered format with clear headings and subheadings.
    - At the top of the ADR section, add a heading "Architecture Decision Records (ADR)". Then, for each ADR, use a subheading with the format ADR number and title, e.g., "ADR 001: Framework Choice".
 
* Use bullet points for lists of requirements, goals, and consequences.
* Cover all major architectural decisions (framework, ML model, database, security, observability, model transparency, API design, etc.).
* Additionally, generate a technical decision table in markdown with the format: | Rule | Signal | Decision | Justification |
* Ensure all content is strictly aligned to the PRD and detailed enough for development guidance.
* Include markdown formatting with appropriate headings and subheadings.
* Include title and table of contents at the top.
* Display the system description at the top.
* Include a summary section at the end with:
- Key decisions made
- Overall architecture overview
* Do not fabricate any information not present in the PRD.
"""

print("*" * 80)
print("Generating ADR for", app_name)
adr_from_template_output = get_completion(adr_prompt, client, model_name, api_provider,temperature=0.3)
adr_from_template_output = clean_llm_output(adr_from_template_output,language="markdown")
print(adr_from_template_output)
if not adr_from_template_output:
    raise ValueError("No ADR output generated")
else:
    save_artifact(adr_from_template_output, "artifacts/convisoft_adr.md", overwrite=True)
    print("ADR generated successfully, saving to artifacts...")

print("*" * 80)

In [ ]:
# Generate a PlantUML component/context diagram.
# Follow the pro-tip above to structure your prompt.
import sys
import os

# Ensure the custom utils.py is imported from your project root
if project_root not in sys.path:
	sys.path.insert(0, project_root)

from utils import load_artifact, render_plantuml_diagram

prd_md_artifact=load_artifact("artifacts/convisoft_prd.md")
database_schema_sql=load_artifact("artifacts/convisoft_schema.sql")

context_diagram_prompt = """
{prompt_template}
You are an expert system architect. 
Generate PlantUML code for a component diagram using the following information.
  - Contracting Visualization Software Product Requirements Document (PRD) Excerpt: {prd_md_artifact}
  - Database Schema: {database_schema_sql}
  - ADRs: {adr_md_artifact}

  Ensure the diagram includes:
  - All major components of the system
  - Relationships and interactions between components
  - External systems and users interacting with the system
  - Data flow between components
  - Ensure that it is 100% strictly aligned to the PRD, ADRs, and database schema.
"""

print("*" * 80)
print("--- Generating Component Diagram ---")
context_puml_raw = get_completion(context_diagram_prompt, client, model_name, api_provider)
context_puml = clean_llm_output(context_puml_raw, language='plantuml')

print("\n--- Generated PlantUML Code ---")
print(context_puml)

# Render the diagram
if context_puml:
	try:
		render_plantuml_diagram(context_puml, "artifacts/convisoft_context_diagram.png")
		print("--- Context Diagram Rendered Successfully ---")
		print("Saved to artifacts/convisoft_context_diagram.png")
	except Exception as e:
		print(f"PlantUML rendering failed: {str(e)}")

print("*" * 80)

In [ ]:
# Generate a PlantUML sequence diagram.
adr_md_artifact = load_artifact("artifacts/convisoft_adr.md")

sequence_diagram_prompt = f"""
You are an expert system architect. 
Your task is to generate PlantUML code for a sequence diagram that strictly follows the requirements and architecture
described in the Contracting Visualization Software PRD and ADR. The diagram should illustrate the process of a user
querying contract data through the dashboard, including authentication, filtering, data retrieval, and visualization.
Only include actors, components, and interactions explicitly described in the PRD, ADR, and database schema.
 - Contracting Visualization Software Product Requirements Document (PRD) Excerpt: {prd_md_artifact}
 - Database Schema: {database_schema_sql}
 - ADRs: {adr_md_artifact}

Context:
* Actors: Contract Analyst, Business Development Manager, Executive/Manager, Data Scientist, Auditor
* Components: Dashboard UI, RESTful API, Authentication Service, Data Filter/Search Service, Contract Data Store, Visualization Engine, Export Module
* Data Source: US Gov Data Sources (FPDS, sam.gov, GSA.gov)

Instructions:
* Show the sequence of a user logging in, submitting a filter/search request, authenticating, retrieving contract data, and visualizing results.
* Include error handling for authentication failure and no results found.
* Use only the terminology and boundaries defined in the PRD, ADR, and database schema.
* Ensure that it is 100% strictly aligned to the PRD, ADR, and database schema.
* Output only valid PlantUML code for the sequence diagram.

"""

print("*" * 80)
print("--- Generating Sequence Diagram ---")
sequence_puml_raw = get_completion(sequence_diagram_prompt, client, model_name, api_provider)
sequence_puml = clean_llm_output(sequence_puml_raw, language='plantuml')

print("\n--- Generated PlantUML Code ---")
print(sequence_puml)

# Render the diagram
if sequence_puml:
	try:
		render_plantuml_diagram(sequence_puml, "artifacts/convisoft_sequence_diagram.png")
		print("--- Sequence Diagram Rendered Successfully ---")
		print("Saved to artifacts/convisoft_sequence_diagram.png")
	except Exception as e:
		print(f"PlantUML rendering failed: {str(e)}")

print("*" * 80)

In [ ]:
# Generate a PlantUML ERD diagram.
erd_diagram_prompt = f"""
You are an expert system architect.

Your task is to generate PlantUML code for an Entity Relationship Diagram (ERD) that strictly follows
the requirements and architecture described in the Contracting Visualization Software PRD and ADR. 
The diagram should include only the entities, attributes, and relationships explicitly defined in the
PRD, ADR, and database schema.
- Contracting Visualization Software Product Requirements Document (PRD) Excerpt: {prd_md_artifact}
- Database Schema: {database_schema_sql}
- ADRs: {adr_md_artifact}

Context:
* Entities: Include all relevant entities (e.g., Contract, User, Company, Role, Location, NAICSCode, PSCCode, Export, AI_recommendation, AI_explanation).
* Attributes: As defined in the database schema (e.g., contract_id, contract_name, company_awarded, date_awarded, location, value, length, naics_code, psc_code, user_id, name, email, role, export_id, export_type, export_date, etc.)
* Relationships: Users initiate exports, contracts are exported, etc.

Instructions:
* Show all relevant fields for each entity as described in the PRD.
* Show foreign key relationships (e.g., Export references User and Contract).
* Use only the terminology and boundaries defined in the PRD, ADR, and database schema.
* Ensure that it is 100% strictly aligned to the PRD, ADR, and database schema.
* Output only valid PlantUML code for the ERD.

Ensure that is 100% strictly aligned to the PRD and ADR.

"""

print("*" * 80)
print("--- Generating ERD Diagram ---")
erd_puml_raw = get_completion(erd_diagram_prompt, client, model_name, api_provider,temperature=0)
erd_puml = clean_llm_output(erd_puml_raw, language='plantuml')

print("\n--- Generated PlantUML Code ---")
print(erd_puml)

# Render the diagram
if erd_puml:
	try:
		render_plantuml_diagram(erd_puml, "artifacts/convisoft_erd_diagram.png")
		print("--- ERD Diagram Rendered Successfully ---")
		print("Saved to artifacts/convisoft_erd_diagram.png")
	except Exception as e:
		print(f"PlantUML rendering failed: {str(e)}")

print("*" * 80)

In [ ]:
# Create the architecture.md file that includes all diagrams and links to PRD and ADR
# Read the SQL file
# Path to your SQL file
sql_path = "c:\\Users\\labadmin\\Documents\\AG-AISOFTDEV\\artifacts\\convisoft_schema.sql"

# Read the SQL file in chunks
def read_sql_in_chunks(file_path, chunk_size=2000):
    with open(file_path, "r", encoding="utf-8") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            yield chunk

# Concatenate all chunks into one string
sql_chunks = list(read_sql_in_chunks(sql_path))
database_schema_sql = "".join(sql_chunks)

# Format as markdown code block
schema_markdown_block = f"""```sql
{database_schema_sql}
```"""

# Print the markdown block to verify
print(schema_markdown_block)

architecture_md_prompt = f"""
You are an expert technical writer and architect.

Generate a comprehensive architecture.md file for the Contracting Visualization Software project. 
The file should include:
- Table of contents.
- A title and brief introduction. Title: "Contracting Visualization Software - Design and Architecture Overview"
- Include the following sections:
   - System overview and description.
   - Key components and their roles.
   - A brief description of each diagram and its purpose.
      - ERD Diagram: Illustrates the data model, entities, attributes, and relationships.
      - Context Diagram: Shows the system's interactions with external entities.
      - Sequence Diagram: Depicts the flow of messages between components.
      - Database Schema: SQL code defining tables, fields, and relationships.
        {schema_markdown_block}
   - Embedded links to the ERD, Context, Sequence diagrams, and Schema (use markdown image syntax).
      ![ERD Diagram](./convisoft_erd_diagram.png)
      ![Context Diagram](./convisoft_context_diagram.png)
      ![Sequence Diagram](./convisoft_sequence_diagram.png)
   - Ensure proper markdown formatting with headings, subheadings, and bullet points.
   - Links to the PRD and ADR markdown files.
      - [Product Requirements Document (PRD)](./convisoft_prd.md)
      - [Architecture Decision Records (ADR)](./convisoft_adr.md)
- Add a Notes section
   - Make sure the image files are in the same directory as `architecture.md` or adjust the paths accordingly.
   - If viewing in VS Code or GitHub, images and links should render if the paths are correct and files exist.
- Any additional notes or summary that would help a developer or stakeholder understand the architecture at a glance.

Use clear markdown formatting, section headings, and concise descriptions.
"""

print("*" * 80)
print("Generating architecture.md file that includes all diagrams and links to PRD and ADR for", app_name)
architecture_md_content = get_completion(architecture_md_prompt, client, model_name, api_provider)
architecture_md = clean_llm_output(architecture_md_content,language='markdown')
print(architecture_md)

if not architecture_md:
    raise ValueError("No architecture.md content generated")
else:
    save_artifact(architecture_md, "artifacts/architecture.md", overwrite=True)
    print("architecture.md generated successfully, saving to artifacts...")

print("*" * 80)